In [2]:
from transformers import BartForConditionalGeneration, BartTokenizer ,PreTrainedTokenizerFast
import json
import re
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [3]:
with open("./Similar_first_input_label_data.json","r",encoding="utf-8") as f:
    data = json.load(f)

print(data["data"][0].keys())

dict_keys(['para'])


In [4]:
print(data["data"][0]["para"][0]["label"][0]["token_str"])

거미줄


In [5]:
Similar_raw = []

for item in data["data"]:
    for para in item["para"]:
        context = para["context"]
        for labels in para["label"]:
            label = labels["sequence"]
            token_str = labels["token_str"]
            Similar_raw.append((context,label,token_str))

print(len(Similar_raw))

64645


In [7]:
train_data_raw , test_data_raw = train_test_split(Similar_raw,test_size = 0.1)

print(len(train_data_raw),len(test_data_raw))

58180 6465


In [9]:
train_data = pd.DataFrame(train_data_raw)
test_data = pd.DataFrame(test_data_raw)
print(len(train_data))
train_data.rename(columns={0: 'context',1:'label',2:'token'}, inplace=True)
test_data.rename(columns={0: 'context',1:'label',2:'token'}, inplace=True)

train_data.head(5)

58180


,context,label,token
0,미친 사람같이 대귈뜰에 매 부르는 소리를 하면서 돌아다녔다.,미친 거 같이 매 부르는 소리를 하면서 돌아다녔다.,거
1,만두 같은 달이건만 보는 이의 심경에 따라 기쁘기도 하고 슬프기도 한격이라고 할는지요.,아마 같은 달이건만 보는 이의 심경에 따라 기쁘기도 하고 슬프기도 한격이라고 할는지요.,아마
2,가을 산골 벼가 금같이 누러니 저녁놀이 자주 같이 붉으니 아버지 살결이 유달리 희다,가을 산골 벼가 늘 같이 누러니 저녁놀이 자주 같이 붉으니 아버지 살결이 유달리 희다,늘
3,너는 돼지처럼 기름지냐.,너는 너 처럼 기름지냐.,너
4,그의 몸은 사시나무와 같이 떨렸다.,그의 몸은 공기 와 같이 떨렸다.,공기


In [10]:
test_data.head(5)

,context,label,token
0,그는 은실 금실로 수놓은 끝동 소매를 조금 치켜서 옥 같은 손으로 뱃전을 짚고 그 ...,그는 은실 금실로 수놓은 끝동 소매를 조금 치켜서 고사리 같은 손으로 뱃전을 짚고 ...,고사리
1,"그의 명성이 떨치고, 문하 구름같이 모여들었다.","그의 명성이 떨치고, 문하 하나 같이 모여들었다.",하나
2,창수는 혼자 이렇게 중얼대며 신작로 넓은 길에 활 등처럼 굽은 S 산 밑 송림 사이...,창수는 혼자 이렇게 중얼대며 신작로 넓은 길에 활 다리 처럼 굽은 S 산 밑 송림 ...,다리
3,"이슬방울이 우수수 떨어지며, 흙 새에 끼었던 흰 모래알이 의붓자식처럼 한 귀퉁이에 ...","이슬방울이 우수수 떨어지며, 흙 새에 끼었던 흰 모래알이 의붓 조각 처럼 한 귀퉁이...",조각
4,탄환처럼도 의장하려라!,이 처럼도 의장하려라!,이


In [11]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2',)
print(tokenizer.special_tokens_map)

print(tokenizer.tokenize(test_data["context"][0]))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


{'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}
['▁그는', '▁은', '실', '▁금', '실로', '▁수', '놓은', '▁끝', '동', '▁소', '매를', '▁조금', '▁치', '켜', '서', '▁옥', '▁같은', '▁손으로', '▁뱃', '전을', '▁짚', '고', '▁그', '▁날', '씬', '한', '▁허', '리를', '▁반', '나마', '▁배', '▁밖으로', '▁기울', '였다.', '▁"']


In [12]:
BOS = tokenizer.bos_token
EOS = tokenizer.eos_token
PAD = tokenizer.pad_token

In [13]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import re
#데이터 처리 클래스
class Similar_Dataset(Dataset):
    def __init__(self, chats, max_len=1024):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.bos= BOS
        self.eos = EOS
        self.pad = PAD
        self.tokenizer = tokenizer
    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한데이터를 차례차례 DataLoader로 넘김
        turn = self._data.iloc[idx]
        l = turn["label"]  # label을 가져온다.
        c = turn["context"]  # 문맥을 가져온다.
        
        c_toked = self.tokenizer.tokenize(self.bos + c + self.eos)
        c_len = len(c_toked)
        
        l_toked = self.tokenizer.tokenize(self.bos + l + self.eos)
        l_len = len(l_toked)

        #최대길이 초과시 처리
        if c_len> self.max_len:
            c_len = self.max_len
            c_toked = c_toked[:c_len]
       
        if l_len> self.max_len:
            l_len = self.max_len
            l_toked = l_toked[:l_len]

        labels_ids = self.tokenizer.convert_tokens_to_ids(l_toked)        
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        decoder_ids = self.tokenizer.convert_tokens_to_ids(l_toked)    
        
        while len(decoder_ids) < self.max_len:
            decoder_ids += [self.tokenizer.pad_token_id]
        
        token_ids = self.tokenizer.convert_tokens_to_ids(c_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        return (token_ids, decoder_ids, labels_ids)

In [14]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    decoder_input = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(decoder_input), torch.LongTensor(label)

In [15]:
train_set = Similar_Dataset(train_data, max_len=1024)
loader = DataLoader(train_set, batch_size=4,  num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [16]:
len(loader)

14545

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [18]:
for i, batch in enumerate(loader):
    token_ids, decoder_ids, labels_ids = batch
    print(f"Batch {i+1}")
    print("Token IDs:", token_ids)
    print(len(token_ids[0]))
    print(token_ids.shape)
    print(token_ids[0][7])
    print(tokenizer.decode(token_ids[0][7]))
    print(tokenizer.decode(token_ids[0]))

    print(len(token_ids))
    print("Decoder_ids:", decoder_ids)
    print("Decoder_ids:", (tokenizer.decode(decoder_ids[0])))

    print("Labels IDs:", labels_ids)
    print("Labels IDs:", (tokenizer.decode(labels_ids[0])))
    break

Batch 1
Token IDs: tensor([[    1, 18769, 14949,  ...,     3,     3,     3],
        [    1, 14306, 10897,  ...,     3,     3,     3],
        [    1, 22505, 10830,  ...,     3,     3,     3],
        [    1, 16938, 14836,  ...,     3,     3,     3]])
1024
torch.Size([4, 1024])
tensor(19089)
가고
</s> 그렇다면 왕은 수렵이라도 가고 궁전만은 비어 있는 것일까 하고 돌축을 하나하나 밟아 가면 또다시 기다란 줄 행랑이 축을 하나하나 밟아 가면 또다시 기다란 줄 행랑이 있는 것이고, 그것을 오른편으로 돌아들어 왼편으로 보이는 별실은 서재인 듯 조용한 목에 뜰 앞에 조롱들 속에서 빛깔 다른 새들이 시스마금 낯선 손님을 맞아 아는 체하고 재재거린다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [19]:
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')
model.to(device)
model.train()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNor

In [20]:
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm

# 모델과 옵티마이저 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 스케쥴러 설정
epochs = 15
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(loader)*epochs)

# 학습 루프
model.train()
for epoch in range(epochs):
    total_loss = 0
    progress_bar = tqdm(loader, desc="Epoch {:1d}".format(epoch+1), leave=False, disable=False)
    for batch in progress_bar:
        batch = tuple(t.to(device) for t in batch)
        input_ids, decoder_input_ids, labels = batch

        # 모델 forward 및 loss 계산
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss

        # 백워드 및 옵티마이저 스텝
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

    print("Epoch {}/{} - Loss: {:.4f}".format(epoch+1, epochs, total_loss / len(loader)))

/home/work/.local/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|          | 0/14545 [00:00<?, ?it/s]

In [18]:
model.save_pretrained("./Similar_bart_model_05_31_last")
tokenizer.save_pretrained("./Similar_bart_model_05_31_last")

('./Similar_bart_model_05_31_last/tokenizer_config.json',
 './Similar_bart_model_05_31_last/special_tokens_map.json',
 './Similar_bart_model_05_31_last/tokenizer.json')

In [31]:
from transformers import GPT2LMHeadModel,PreTrainedTokenizerFast,AutoModelForCausalLM, AutoTokenizer
def generate_sequence(context, model, tokenizer, device, num_sequences=2):
      # 모델을 평가 모드로 설정

    # 입력 문맥 토크나이징
    input_ids = tokenizer.encode(BOS + context + EOS, return_tensors="pt").to(device)

    # 모델을 사용하여 출력 생성
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=512,
        temperature=0.7,
        top_k=5,
        top_p=0.9,
        repetition_penalty=2.0,
        do_sample=True,
        num_return_sequences=num_sequences
    )

    generated_sequences = []
    for generated_sequence in output_sequences:
        # 생성된 시퀀스를 디코딩하여 텍스트로 변환
        decoded_text = tokenizer.decode(generated_sequence, skip_special_tokens=True)
        generated_sequences.append(decoded_text)
    
    return generated_sequences

model_path = "./"+"Similar_gpt2_model1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def multiple_replace(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

replacements = {BOS: "", EOS: "", PAD: ""}

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
model.eval()
context = test_data["context"][3]
print("원본:", context)

# context를 input으로 넣으면 context +question +answer 형태로 만듬
generated_sentences = generate_sequence(context, model, tokenizer, device, num_sequences=5)

for idx, sentence in enumerate(generated_sentences):
    sentence = multiple_replace(sentence, replacements)
    print(f"생성 {idx+1}:", sentence)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


원본: 나는 이 잔디밭에 누어서 소년 다운 온갖 공상을 푸른 하늘로 날개처럼 펼치었었소.
생성 1: 나는 이 잔디밭에 누어서 소년 다운 온갖 공상을 푸른 하늘로 날개처럼 펼치었었소..었다.습니다. 였요.이었다소 말았으니하였이지말정말로언 언 하였정이말이 말이언이언은 말은은 말을 은 말도도 뜻밖이 소리소리가소리 소리가 비비 비가비가비는비를며비와비,비고비고비고비고고글 글 문 글을 써 편지를 써서 글씨 편지 속에 넣어에 넣고 몸에 걸 데니 주머니주머니엔에는에도 들어 바까지 든든들어속속에 들어가 들어갔속으로 들어간다.이다.
생성 2: 나는 이 잔디밭에 누어서 소년 다운 온갖 공상을 푸른 하늘로 날개처럼 펼치었었소.습니다.았 놓쳤습니다니렸으니 다행이지.지....... 소용 " 그러나 그것도 또 이것도것도 그렇지만도 이건 것도 그렇 아니라 황 란 역시 그러 같은 건이란 말 이란 말은란 뜻밖의 말에 앞이 아뜩해지며 아무 말도 할 수가 없더구나? 도무지할 수 없이 식 없는 일 인 충동을동이동 동무를 하고 어이가 있!무가 있는 하는 것 같 보인가요어렴이는하는 이는 누구 시인은 사람은 것은 사람일인 듯싶은갑는 구름가는사람들 은가 자꾸만 아는 사람이 먹는 명 저가, 맛 는다는 좋다는 소문을 듣고 마음에 남의 것을 좋은 좋지 좋고 좋 싫좋 생각이 제일 좋다고 좋을 가장 좋아야겠다는 생각 그 것이 좋아하는 싶은 것일 싶 것인데 것이지 지금 처럼 요즘 우리 것만 가지고 오늘이라도 이것저것을 위해서 나 옛날까 먼저 생긴 것이니 예전라도 이런 데까지 오늘날 우리나라 일본이나 조선, 일본, 나라 것까지 일본으로 해서 무슨지,분 하고, 다른 나라에 분 여러 나라로 않고,나라에하 고 하지와해서 척척시켰다시피부터 놈했하겠거니와뿐입니다.에,한으로,석 애초부터반애당초 당초하였 사러 원수보다도 것입니다.하여 이와연 알에서즉 방금 오직으로 한으로서 책임물 정하였던 바발 발랄 하여튼히 무엇으므로미행하였다.]으로써 미 육 불난 까닭 장차 위하여 이미 말한 사실혼으로서의진대 위에 대로서 무어 야 모적으로든 실

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [33]:
%pip uninstall -y transformers
%pip install git+https://github.com/huggingface/transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: transformers 4.34.0
Uninstalling transformers-4.34.0:
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/lib/python3.10/shutil.py", line 816, in move
    os.rename(src, real_dst)
PermissionError: [Errno 13] Permission denied: '/usr/local/bin/transformers-cli' -> '/tmp/pip-uninstall-rsim9_ne/transformers-cli'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/uninstall.py", line 105, in run
    uninstall_pathset = req.uninstall(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_install.py", line 687, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/req/req_uninstall.py", line 381

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8uzp1br7
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8uzp1br7
  Resolved https://github.com/huggingface/transformers to commit d19566e8523547d833d674d973908dae569dfba4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for huggingface-hub<1.0,>=0.23.0 from https://files.pythonhosted.org/packages/78/71/6ce4136149cb42b98599d49c39b3a39dd6858b5f9307490998c40e26a51e/huggingface_hub-0.23.2-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.20,>=0.19 from https://files.pythonhosted.org/packages/40/4f/eb78de4af3b17b589f43a369cbf0c3a7173f25c3d2cd93068852c0

In [1]:
from transformers import GPT2LMHeadModel,PreTrainedTokenizerFast,AutoModelForCausalLM, AutoTokenizer
model_path = "./"+"Similar_phi3_model1"

tokenizer =  AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.eval()
Similar_data_test= {"data":[]}
Similar_data_train= {"data":[]}
def Similar_generation(Similar_data, context ,model,tokenizer,device):
    para = {"para":[]}
    sequence = {"sequence":[]}
    sequence["context"] = context
     
    generated_sentences = generate_sequence(context, model, tokenizer, device, num_sequences=2)
    for idx, sentence in enumerate(generated_sentences):
        sentence = multiple_replace(sentence, replacements)
        sequence["sequence"].append(sentence)

    para["para"].append(sequence)
    Similar_data["data"].append(para)

    return Similar_data
    


for i, data in tqdm(enumerate(test_data["context"][:1000]), total=test_data["context"][:1000].shape[0]):
    Similar_generation(Similar_data_test, data, model, tokenizer, device)

with open(model_path+'_testdata.json', 'w', encoding='utf-8') as file :
    json.dump(Similar_data_test, file, indent='\t', ensure_ascii=False)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

NameError: name 'device' is not defined

In [ ]:

for i, data in tqdm(enumerate(train_data["context"][:1000]), total=train_data["context"][:1000].shape[0]):
    Similar_generation(Similar_data_train, data, model, tokenizer, device)
    
with open(model_path+'_traindata.json', 'w', encoding='utf-8') as file :
    json.dump(Similar_data_train, file, indent='\t', ensure_ascii=False)

100%|██████████| 1000/1000 [01:23<00:00, 12.00it/s]
